<a href="https://colab.research.google.com/github/harshinikoppula30/intelliwaste_analyzer/blob/main/intelliwaste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.chdir("/content/drive/MyDrive/Garbage_Classification_Project/Garbage-Classification-System")


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!ls /content


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ✅ Step 1: Create ImageDataGenerator with validation split
gen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=(0.5, 1.5),
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode='nearest',
    validation_split=0.2  # 👈 80% train, 20% val
)

# ✅ Step 2: Training data (80%)
train_data = gen.flow_from_directory(
    '/content/drive/MyDrive/Garbage_Classification_Project/Garbage-Classification-System/full_garbage_dataset/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    seed=42,
    shuffle=True
)

# ✅ Step 3: Validation data (20%)
val_data = gen.flow_from_directory(
    '/content/drive/MyDrive/Garbage_Classification_Project/Garbage-Classification-System/full_garbage_dataset/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    seed=42,
    shuffle=True
)




In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

vgg16 = VGG16(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

# Optionally fine-tune last 4 layers
for layer in vgg16.layers[:-4]:
    layer.trainable = False
for layer in vgg16.layers[-4:]:
    layer.trainable = True

# Build custom head
x = layers.GlobalAveragePooling2D()(vgg16.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
prediction = layers.Dense(12, activation='softmax')(x)

# Create model
model = models.Model(inputs=vgg16.input, outputs=prediction)

# Compile
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
result = model.fit(
    train_data,
    validation_data=val_data,               # Keep 20% for validation
    epochs=15,                              # 👉 Train for only 15 epochs
    steps_per_epoch=len(train_data),
    validation_steps=len(val_data),
    verbose=1
)



In [ ]:
# Save your trained model to Google Drive
model.save("/content/drive/MyDrive/waste_classifier_full_vgg16.keras")
print("✅ Model saved successfully!")

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/waste_classifier_full_vgg16.keras")
print("✅ Model loaded successfully!")


In [ ]:
output_class = [
    "battery", "biological", "brown-glass", "cardboard", "clothes", "green-glass",
    "metal", "paper", "plastic", "shoes", "trash", "white-glass"
]

In [ ]:
def waste_prediction(new_image):
    test_image = image.load_img(new_image, target_size=(224, 224))
    plt.axis("off")
    plt.imshow(test_image)
    plt.show()

    test_image = image.img_to_array(test_image) / 255
    test_image = np.expand_dims(test_image, axis=0)

    predicted_array = model.predict(test_image)
    predicted_value = output_class[np.argmax(predicted_array)]
    predicted_accuracy = round(np.max(predicted_array) * 100, 2)

    print("Your waste material is", predicted_value, "with", predicted_accuracy, "% accuracy.")


In [ ]:
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt


In [ ]:
waste_prediction("/content/drive/MyDrive/Garbage_Classification_Project/Garbage-Classification-System/test_images/biological_biological212.jpg")


In [ ]:
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
plt.plot(result.history["accuracy"], label="Training Accuracy", color='green')
plt.title("Training Accuracy Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(result.history["loss"], label="Training Loss", color='red')
plt.title("Training Loss Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
model.save("/content/drive/MyDrive/waste_classifier_vgg16.keras")


In [ ]:
from tensorflow.keras.models import load_model
model = load_model("/content/drive/MyDrive/waste_classifier_vgg16.keras")


In [ ]:
import os
import shutil


test_dir = "test_images"
if os.path.exists(test_dir):
    shutil.rmtree(test_dir)
    print("🧹 Old test_images folder removed.")

os.makedirs(test_dir)
print("✅ New test_images folder created.")

# Step 3: Define class labels
output_class = [
    'battery', 'biological', 'brown-glass', 'cardboard', 'clothes',
    'green-glass', 'metal', 'paper', 'plastic', 'shoes', 'trash', 'white-glass'
]


source_dir = "/content/drive/MyDrive/Garbage_Classification_Project/Garbage-Classification-System/full_garbage_dataset/"

for cls in output_class:
    class_path = os.path.join(source_dir, cls)

    if not os.path.exists(class_path):
        print(f"⚠️ Skipped (folder not found): {cls}")
        continue

    images = [img for img in os.listdir(class_path) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]

    for img_file in images[:5]:
        src = os.path.join(class_path, img_file)
        dst = os.path.join(test_dir, f"{cls}_{img_file}")
        shutil.copy(src, dst)



In [ ]:
import os

test_dir = "test_images"

test_images = []
true_labels = []

output_class = ['battery', 'biological', 'brown-glass', 'cardboard', 'clothes',
                'green-glass', 'metal', 'paper', 'plastic', 'shoes', 'trash', 'white-glass']

# Manual fallback label mapping
label_map = {
    "brownglass": "brown-glass",
    "greenglass": "green-glass"
}

for file in os.listdir(test_dir):
    if file.endswith(".jpg"):
        test_images.append(file)
        matched_label = None
        # Try exact match from output_class
        for cls in output_class:
            if cls in file:
                matched_label = cls
                break
        # Try fallback mapping if no match
        if not matched_label:
            for key, value in label_map.items():
                if key in file:
                    matched_label = value
                    break
        if matched_label:
            true_labels.append(matched_label)
        else:
            print(f"⚠️ Skipped file (no label matched): {file}")


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

predicted_labels = []

for img_file in test_images:
    img_path = os.path.join(test_dir, img_file)
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = model.predict(img_array)
    pred_class = output_class[np.argmax(pred)]
    predicted_labels.append(pred_class)

# 🧠 Check what was predicted and what was true
print("✅ True Labels:", true_labels)
print("🔮 Predicted Labels:", predicted_labels)

# 📊 Confusion Matrix
cm = confusion_matrix(true_labels, predicted_labels, labels=output_class)

plt.figure(figsize=(14, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=output_class, yticklabels=output_class)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

# 📄 Classification Report
print("\n📋 Classification Report:\n")
print(classification_report(true_labels, predicted_labels, labels=output_class))
